In [2]:
from pyspark.context import SparkContext
from pyspark.sql.context import SQLContext
from pyspark.sql.session import SparkSession

In [3]:
sc = SparkContext()
sqlContext = SQLContext(sc)
spark = SparkSession(sc)

In [5]:
import re
import pandas as pd
import glob

In [8]:
data_files = glob.glob('*.gz')
df = spark.read.text(data_files)

In [28]:
host_pattern = r'(^\S+\.[\S+\.]+\S+)\s'
ts_pattern = r'\[(\d{2}/\w{3}/\d{4}:\d{2}:\d{2}:\d{2} -\d{4})]'
method_uri_protocol_pattern = r'\"(\S+)\s(\S+)\s*(\S*)\"'
status_pattern = r'\s(\d{3})\s'
content_size_pattern = r'\s(\d+)$'

In [24]:
from pyspark.sql.functions import regexp_extract

In [57]:
df_logs = df.select(regexp_extract('value', host_pattern, 1).alias('host'),
                   regexp_extract('value', ts_pattern, 1).alias('timestamp'),
                   regexp_extract('value', method_uri_protocol_pattern, 1).alias('method'),
                   regexp_extract('value', method_uri_protocol_pattern, 2).alias('endpoint'),
                   regexp_extract('value', method_uri_protocol_pattern, 3).alias('protocol'),
                   regexp_extract('value', status_pattern, 1).cast('integer').alias('status'),
                   regexp_extract('value', content_size_pattern, 1).cast('integer').alias('content_size'))

In [58]:
df_logs.take(5)

[Row(host='199.72.81.55', timestamp='01/Jul/1995:00:00:01 -0400', method='GET', endpoint='/history/apollo/', protocol='HTTP/1.0', status=200, content_size=6245),
 Row(host='unicomp6.unicomp.net', timestamp='01/Jul/1995:00:00:06 -0400', method='GET', endpoint='/shuttle/countdown/', protocol='HTTP/1.0', status=200, content_size=3985),
 Row(host='199.120.110.21', timestamp='01/Jul/1995:00:00:09 -0400', method='GET', endpoint='/shuttle/missions/sts-73/mission-sts-73.html', protocol='HTTP/1.0', status=200, content_size=4085),
 Row(host='burger.letters.com', timestamp='01/Jul/1995:00:00:11 -0400', method='GET', endpoint='/shuttle/countdown/liftoff.html', protocol='HTTP/1.0', status=304, content_size=0),
 Row(host='199.120.110.21', timestamp='01/Jul/1995:00:00:11 -0400', method='GET', endpoint='/shuttle/missions/sts-73/sts-73-patch-small.gif', protocol='HTTP/1.0', status=200, content_size=4179)]